## -- [Day 12: Rain Risk](https://adventofcode.com/2020/day/12) --


    Action N means to move north by the given value.
    Action S means to move south by the given value.
    Action E means to move east by the given value.
    Action W means to move west by the given value.
    Action L means to turn left the given number of degrees.
    Action R means to turn right the given number of degrees.
    Action F means to move forward by the given value in the direction the ship is currently facing.


In [1]:
from enum import Enum

INPUT_FILE = 'input_d12.txt'
EXAMPLE_FILE = 'input2_d12.txt'

class Direction(Enum):
    N = 0
    E = 90
    S = 180
    W = 270
    
    def rotate(self, right_left, degree):
        '''
        Returns the new direction after rotating a set number of degrees
        :param right_left: either 'R' for clockwise (the default) turn or 'L' for counter-clockwise
        :param degree: The number degrees to turn
        '''
        if degree % 90 != 0:
            raise IllegalArgument('Can only turn in multiples of 90 degrees')
        if right_left == 'L':
            degree = -degree
        new_dir = (self.value + degree) % 360
        return Direction(new_dir)

class Ship:
    def __init__(self, input_file=INPUT_FILE):
        # Initial direction is given as EAST
        self.orientation = Direction.E
        self.east_west = 0
        self.north_south = 0
        self.load_directions(input_file)
    
    def process_action(self, command, param):
        '''
        Processes an individual action
        :param command: One of [N, S, E, W, L, R, F]
        :param param: The parameter--units to move when [N,S,E,W,F], or degrees with [R,L]
        '''
        if command in ['R','L']:
            self.orientation = self.orientation.rotate(command, param)
            print(f'Rotating. Now facing {self.orientation}')
        elif command == 'F':
            self.move(self.orientation, param)
        elif command in ['N','S','E','W']:
            self.move(Direction[command], param)
        else:
            raise IllegalArgument('Unrecognized command')
    
    def move(self, direction, distance):
        '''
        For a move in a given direction, updates state accordingly
        :param direction: a Direction enum
        :param distance: the integer distance to move
        '''
        if direction == Direction.N:
            self.north_south += distance
        elif direction == Direction.S:
            self.north_south -= distance
        elif direction == Direction.E:
            self.east_west += distance
        elif direction == Direction.W:
            self.east_west -= distance
        else:
            raise IllegalArgument('Illegal direction')
            
        print(f'Move {direction} by {distance}, now at {self.north_south}, {self.east_west}')
        
    def manhattan_dist(self):
        '''
        Part 1 solution - returns the Manhattan distance from the origin
        '''
        return abs(self.north_south) + abs(self.east_west)
        
    def load_directions(self, input_file):
        with open(input_file) as fh:
            for line in fh.readlines():
                self.process_action(line[0], int(line[1:]))

In [2]:
ex = Ship(EXAMPLE_FILE)
ex.manhattan_dist()

Move Direction.E by 10, now at 0, 10
Move Direction.N by 3, now at 3, 10
Move Direction.E by 7, now at 3, 17
Rotating. Now facing Direction.S
Move Direction.S by 11, now at -8, 17


25

In [3]:
# Part 1 solution
ship = Ship()
ship.manhattan_dist()

Move Direction.E by 5, now at 0, 5
Rotating. Now facing Direction.S
Move Direction.E by 5, now at 0, 10
Rotating. Now facing Direction.E
Move Direction.E by 80, now at 0, 90
Rotating. Now facing Direction.N
Move Direction.E by 3, now at 0, 93
Move Direction.N by 5, now at 5, 93
Move Direction.N by 10, now at 15, 93
Move Direction.S by 2, now at 13, 93
Move Direction.W by 1, now at 13, 92
Move Direction.N by 98, now at 111, 92
Rotating. Now facing Direction.S
Move Direction.W by 1, now at 111, 91
Move Direction.S by 55, now at 56, 91
Rotating. Now facing Direction.E
Move Direction.E by 73, now at 56, 164
Move Direction.N by 1, now at 57, 164
Move Direction.E by 3, now at 57, 167
Move Direction.S by 2, now at 55, 167
Move Direction.E by 5, now at 55, 172
Rotating. Now facing Direction.S
Move Direction.E by 2, now at 55, 174
Move Direction.S by 4, now at 51, 174
Move Direction.S by 34, now at 17, 174
Rotating. Now facing Direction.W
Move Direction.W by 2, now at 17, 172
Rotating. Now faci

1565

### Part 2
Almost all of the actions indicate how to move a waypoint which is relative to the ship's position:

    Action N means to move the waypoint north by the given value.
    Action S means to move the waypoint south by the given value.
    Action E means to move the waypoint east by the given value.
    Action W means to move the waypoint west by the given value.
    Action L means to rotate the waypoint around the ship left (counter-clockwise) the given number of degrees.
    Action R means to rotate the waypoint around the ship right (clockwise) the given number of degrees.
    Action F means to move forward to the waypoint a number of times equal to the given value.

The waypoint starts 10 units east and 1 unit north relative to the ship. The waypoint is relative to the ship; that is, if the ship moves, the waypoint moves with it.

In [4]:
# Refactor of everything above
from enum import Enum

INPUT_FILE = 'input_d12.txt'
EXAMPLE_FILE = 'input2_d12.txt'

class Direction(Enum):
    N = 0
    E = 90
    S = 180
    W = 270
        
class Point:
    '''
    A Point has 2D coordinates on the X (east-west) and Y (north-south) axes
    '''
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def move(self, direction, distance):
        '''
        For a move in a given direction, updates state accordingly
        :param direction: a Direction enum
        :param distance: the integer distance to move
        '''
        if direction == Direction.N:
            self.y += distance
        elif direction == Direction.S:
            self.y -= distance
        elif direction == Direction.E:
            self.x += distance
        elif direction == Direction.W:
            self.x -= distance
        else:
            raise IllegalArgument('Illegal direction')

    def move_x_y(self, dxdy):
        '''
        Moves the coordinates by the amounts specified
        :param dxdy: tuple of the amount to move in the x and y axes
        '''
        self.x += dxdy[0]
        self.y += dxdy[1]
        
    def manhattan(self):
        '''
        :return: The Manhattan distance relative to the origin
        '''
        return abs(self.x) + abs(self.y)

    def __repr__(self):
        return f'<{self.__class__} ({self.x}, {self.y})>'

class Waypoint(Point):
    '''
    A Waypoint is a Point that has the ability to rotate around its origin
    '''
    def rotate(self, right_left, degree):
        '''
        Rotates the waypoint around the origin in 90-degree increments
        :param right_left: 'R' for clockwise or 'L' for counter-clockwise
        :param degree: number of degrees to rotate (in 90-degree steps)
        '''
        num_turns = (degree // 90) % 4
        if right_left == 'L' and num_turns != 0:
            num_turns = 4 - num_turns
        for _ in range(num_turns):
            self.x, self.y = self.y, -self.x
            
    def __mul__(self, other):
        if isinstance(other, int):
            return (other * self.x, other * self.y)
        else:
            raise NotImplemented
                
class NavSystem:
    '''
    NavSystem has a Point representing ship location and a Waypoint
    vector from the ship
    '''
    
    def __init__(self, wp_x=10, wp_y=1):
        '''
        :param wp_x: Starting x position for waypoint relative to ship
        :param wp_y: Starting y position for waypoint relative to ship
        '''
        self.waypoint = Waypoint(wp_x, wp_y)
        self.ship = Point(0,0)
        
    def process_action(self, command, param):
        '''
        Processes an individual action according to the revised Part 2 definitions
        :param command: One of [N, S, E, W, L, R, F]
        :param param: The parameter--units to move when [N,S,E,W,F], or degrees with [R,L]
        '''
        if command in ['R','L']:
            self.waypoint.rotate(command, param)
        elif command == 'F':
            self.ship.move_x_y(self.waypoint * param)
        elif command in ['N','S','E','W']:
            self.waypoint.move(Direction[command], param)
        else:
            raise IllegalArgument('Unrecognized command')
            
        self.print_loc(command, param)
            
    def load_actions(self, input_file):
        '''
        Loads actions from a file
        :param input_file: string filename
        '''
        with open(input_file) as fh:
            for line in fh.readlines():
                self.process_action(line[0], int(line[1:]))
                
    def print_loc(self, cmd, param):
        print(f'{cmd} {param}: ship at {self.ship.x, self.ship.y}; waypoint {self.waypoint.x, self.waypoint.y}')

In [5]:
p = Point(10,3)
p.move_x_y((2,2))
assert (p.x, p.y) == (12,5)
assert p.manhattan() == 17

In [6]:
wp = Waypoint(10,3)
assert wp * 3 == (30, 9)
wp.move(Direction.N, 3)
wp.move(Direction.E, 3)
assert wp.x, wp.y == (13,6)
wp.rotate('L', 90)
assert wp.x, wp.y == (6, -13)
wp.rotate('R', 180)
assert wp.x, wp.y == (13, 6)

In [7]:
# Example data test
ex = NavSystem()
ex.load_actions(EXAMPLE_FILE)
assert ex.ship.manhattan() == 286

F 10: ship at (100, 10); waypoint (10, 1)
N 3: ship at (100, 10); waypoint (10, 4)
F 7: ship at (170, 38); waypoint (10, 4)
R 90: ship at (170, 38); waypoint (4, -10)
F 11: ship at (214, -72); waypoint (4, -10)


In [8]:
# Part 2 actual data
nav = NavSystem(10,1)
nav.load_actions(INPUT_FILE)

E 5: ship at (0, 0); waypoint (15, 1)
R 90: ship at (0, 0); waypoint (1, -15)
E 5: ship at (0, 0); waypoint (6, -15)
L 90: ship at (0, 0); waypoint (15, 6)
F 80: ship at (1200, 480); waypoint (15, 6)
L 90: ship at (1200, 480); waypoint (-6, 15)
E 3: ship at (1200, 480); waypoint (-3, 15)
N 5: ship at (1200, 480); waypoint (-3, 20)
F 10: ship at (1170, 680); waypoint (-3, 20)
S 2: ship at (1170, 680); waypoint (-3, 18)
W 1: ship at (1170, 680); waypoint (-4, 18)
F 98: ship at (778, 2444); waypoint (-4, 18)
L 180: ship at (778, 2444); waypoint (4, -18)
W 1: ship at (778, 2444); waypoint (3, -18)
F 55: ship at (943, 1454); waypoint (3, -18)
L 90: ship at (943, 1454); waypoint (18, 3)
F 73: ship at (2257, 1673); waypoint (18, 3)
N 1: ship at (2257, 1673); waypoint (18, 4)
E 3: ship at (2257, 1673); waypoint (21, 4)
S 2: ship at (2257, 1673); waypoint (21, 2)
E 5: ship at (2257, 1673); waypoint (26, 2)
R 90: ship at (2257, 1673); waypoint (2, -26)
E 2: ship at (2257, 1673); waypoint (4, -26

E 4: ship at (-4943, -20747); waypoint (-41, 65)
F 93: ship at (-8756, -14702); waypoint (-41, 65)
S 2: ship at (-8756, -14702); waypoint (-41, 63)
F 10: ship at (-9166, -14072); waypoint (-41, 63)
R 180: ship at (-9166, -14072); waypoint (41, -63)
F 74: ship at (-6132, -18734); waypoint (41, -63)
S 1: ship at (-6132, -18734); waypoint (41, -64)
S 1: ship at (-6132, -18734); waypoint (41, -65)
F 67: ship at (-3385, -23089); waypoint (41, -65)
R 90: ship at (-3385, -23089); waypoint (-65, -41)
E 5: ship at (-3385, -23089); waypoint (-60, -41)
F 66: ship at (-7345, -25795); waypoint (-60, -41)
L 90: ship at (-7345, -25795); waypoint (41, -60)
F 27: ship at (-6238, -27415); waypoint (41, -60)
S 2: ship at (-6238, -27415); waypoint (41, -62)
R 270: ship at (-6238, -27415); waypoint (62, 41)
F 11: ship at (-5556, -26964); waypoint (62, 41)
E 4: ship at (-5556, -26964); waypoint (66, 41)
N 3: ship at (-5556, -26964); waypoint (66, 44)
F 32: ship at (-3444, -25556); waypoint (66, 44)
L 180: s

In [9]:
# Part 2 solution
nav.ship.manhattan()

78883